# API To Multi-Resource Blotto Game 

In [11]:
#First Import the Optimization Script, it has one function build_lp
include("mrblotto_base.jl")
using JuMP, Clp
using GLPK

The script only has one function build_lp.  To build the linear program you must specify:

1. The number of targets
2. The number of each platform for each player
3. The payoff function

Let's sepcift 1 and 2

In [9]:
K = 5 # Number of Targets
N1 = [10,0,0,0,1] # P1 Platforms
N2 = [5,0,0,0,0]; # P2 Platforms

The above cell says that there are 5 targets/battelfields.  Player 1 has 10 platforms of type 1 (given by the index of the number "10" in N1) and 1 platform of type 5, (given by the index of the "1" in N1).  Player 2 just has 5 platforms of type 1.  However, to begin let's just consider the 1 platform type case and change player 1's platforms to reflect that:

In [6]:
N1 = [10,0,0,0,0];

The payoff function is where we will define the relative importance of the platforms and how they "fight" against one another.  However, all payoff functions take 11 inputs.  

1. The first input is the target number.
2. The next 5 inputs are p1's allocations of each platform to the target specified by the first input
3. THe next 5 inputs are p2's allocations of each platform to the target specified by the first input

Let's look at an example

In [19]:
function payoff(k,x1,x2,x3,x4,x5,y1,y2,y3,y4,y5)
    if k == 1
        if x1>y1
            return 1
        elseif x1<y1
            return -1
        else
            return 0
        end
    
    elseif k == 2
        if x1>y1
            return 1
        elseif x1<y1
            return -1
        else
            return 0
        end
                    
    elseif k == 3
        if x1>y1
            return 1
        elseif x1<y1
            return -1
        else
            return 0
        end
                            
    elseif k == 4
        if x1>y1
            return 1
        elseif x1<y1
            return -1
        else
            return 0
        end
                                    
    elseif k == 5
        if x1>y1
            return 1
        elseif x1<y1
            return -1
        else
            return 0
        end
    end
end


payoff (generic function with 1 method)

This payoff function is a special case since the winner of each target is determined the same way.  Specifically, if player 1 puts more of platform of type 1 to the target, player 1 wins.  If player 1 and player 2 put the same number of platforms to the target, they tie.  Otherwise, player 1 loses.  In general, the targets can be heterogeneous.  

We can now show how to solve build and solve the LP

In [29]:
model, p1_utility = build_lp(K, N1, N2, payoff)
optimize!(model)
print("P1's Utility is ", value(p1_utility)) # Because p1_utility is an optimizer variable, we need to call value to see it

P1's Utility is 2.3999999999999995

This is the simple Blotto game but P1 has more platforms than P2, so as expected, P1 has a positive expected utility.  Now suppose we wanted to consider the case where:

1. Player 1 had 1 resource of a capability on a platform
2. Player 2 had 2 resources of a capability on a platform

In english, player 1 has 1 bomb per platform and player 2 has two boms per platoforms and whoever allocates more bombs to a target wins.  We can capture this in a new payoff function:



In [37]:
function payoff_asymmetric_plats(k,x1,x2,x3,x4,x5,y1,y2,y3,y4,y5)
    if k == 1
        if x1>2*y1
            return 1
        elseif x1<2*y1
            return -1
        else
            return 0
        end
    
    elseif k == 2
        if x1>2*y1
            return 1
        elseif x1<2*y1
            return -1
        else
            return 0
        end
                    
    elseif k == 3
        if x1>2*y1
            return 1
        elseif x1<2*y1
            return -1
        else
            return 0
        end
                            
    elseif k == 4
        if x1>2*y1
            return 1
        elseif x1<2*y1
            return -1
        else
            return 0
        end
                                    
    elseif k == 5
        if x1>2*y1
            return 1
        elseif x1<2*y1
            return -1
        else
            return 0
        end
    end
end

payoff_asymmetric_plats (generic function with 1 method)

Now the payoff says that player 1 wins iff it allocates more bombs to a target, taking into account that player 2 has more bombs per plane.

In [63]:
model2, p1_utility2 = build_lp(K, N1, N2, payoff_asymmetric_plats)
optimize!(model2)
print("P1's Utility is ", value(p1_utility2))

P1's Utility is -1.5987211554602254e-14

So now player 1 earns 0, which means under this parameterization, there is no benefit to fractionation (an interesting experiment is to see how this changes as you change the number of targets).

Finally, suppose we want to examine heterogeneity.  Consider the following scenario:

1. Three targets
2. Target 1 requires capability 1
3. Target 2 requires capability 2
4. Target 3 requires either capability 3 or 4.
5. Player 1 has 10 total platforms, 5 with capability 1 and 5 with capability 2
6. Player 2 has 5 platforms but each platform has both capability 1 and capability 2

First we modify the resource and target parameters

In [68]:
K = 3 # Number of Targets
N1 = [5,5,0,0,0] 
N2 = [5,0,0,0,0]; 

Note that the platform in the first index for player 2 does not necessarily have to have the same physical interpretation as the platform of the first index of player 1.  In other words, the 5 in the first position for player 1 says that player 1 has 5 platforms that just have capability 1.  The 5 in the first position for player 2 says that player 2 has 5 platforms that each have 1 resource of capability 1 and capability 2.  

Here is how you write the payoff function

In [69]:
function payoff_hetero(k,x1,x2,x3,x4,x5,y1,y2,y3,y4,y5)
    if k==1
        if x1<y1
            return -1
        elseif x1>y1
            return 1
        else
            return 0
        end
    elseif k==2
        if x2<y1
            return -1
        elseif x2>y1
            return 1
        else
            return 0
        end
        
    elseif k==3
        if x1+x2<2*y1
            return -1
        elseif x1+x2>2*y1
            return 1
        else
            return 0
        end
    end
end

payoff_hetero (generic function with 1 method)

In [70]:
model3, p1_utility3 = build_lp(K, N1, N2, payoff_hetero)
optimize!(model3)
print("P1's Utility is ", value(p1_utility3))

P1's Utility is 0.9999999999999996

So, even though player 1 and player 2 has the same number of total resources, player 1 does much better because of the heterogeneity of the platforms.  